In [1]:
import os
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=Warning)

from dlc_utils.dlc_config import select_session, get_trial_video_list, clean_pretrained_project
from dlc_utils.dlc_downsample import check_for_downsample, downsample_videos

import deeplabcut

ROOT = '/Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/'
VIDEO_ROOT = os.path.join(ROOT, 'tasks', 'rhAirpuff', '8. Probabilistic_Reward_Airpuff_Choice', 'videos')
SESSION_ROOT = os.path.join(ROOT, 'data', 'processed', 'processed_Probabilistic_Reward_Airpuff_Choice')

# Select session
DATE = '230628'
MONKEY = 'Aragorn'

session_df = select_session(SESSION_ROOT, DATE, MONKEY, correct_only=True)
video_path_list = get_trial_video_list(session_df, VIDEO_ROOT, DATE, MONKEY)

print(f'Number of video files: {len(video_path_list)}')
session_df.head(5)


Loading DLC 2.3.5...
Selecting session: 230628_Aragorn
Checking for video files...
  Video directory: /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230628_Aragorn


100%|██████████| 451/451 [00:00<00:00, 13504.99it/s]

Number of video files: 451
Number of video files: 451


,date,session_num,subject,trial_num,block,condition,correct,error,error_type,behavioral_code_markers,...,blink_raster,blink_raster_window,blink_duration_window,pupil_pre_CS,lick_in_window,blink_in_window,lick_duration,blink_duration_sig,blink_duration_offscreen,eye_distance
0,230628,0,Aragorn,1,1,1,1,0,0,"[9, 100, 101, 102, 103, 104, 105, 106, 107, 10...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.350000,"[7438.0, 7438.0, 7438.0, 7438.0, 7435.0, 7435....",1.0,1.0,0.952000,0.284667,0.496667,144.909405
1,230628,0,Aragorn,2,1,1,1,0,0,"[9, 100, 101, 102, 103, 104, 105, 106, 107, 10...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.172000,"[7154.0, 7145.0, 7145.0, 7145.0, 7145.0, 7143....",1.0,1.0,1.000000,0.022667,0.414000,97.851549
2,230628,0,Aragorn,3,1,1,1,0,0,"[9, 100, 101, 102, 103, 104, 105, 106, 107, 10...",...,"[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.012667,"[6747.0, 6747.0, 6747.0, 6747.0, 6745.0, 6745....",1.0,1.0,1.000000,0.009333,0.420667,93.014037
3,230628,0,Aragorn,4,1,1,1,0,0,"[9, 100, 101, 102, 103, 104, 105, 106, 107, 10...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,"[7000.0, 7000.0, 6988.0, 6988.0, 6988.0, 6988....",1.0,1.0,1.000000,0.000000,0.094667,60.959783
4,230628,0,Aragorn,5,1,1,1,0,0,"[9, 100, 101, 102, 103, 104, 105, 106, 107, 10...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,"[6973.0, 6973.0, 6973.0, 6972.0, 6972.0, 6966....",1.0,1.0,0.686667,0.000000,0.321333,58.929321


In [2]:
import ipywidgets as widgets
from IPython.display import display

model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = widgets.Dropdown(
  options=model_options,
	value=model_options[3],
	description="Choose a DLC ModelZoo model!",
	disabled=True
)
print(f'Model selected: {model_selection.value}')

Model selected: primate_face


In [3]:
from PIL import Image
import cv2

project_name = f'{DATE}_{MONKEY}'
your_name = 'rahim'
model2use = model_selection.value
# .mp4 or .avi etc.
videotype = os.path.splitext(video_path_list[0])[-1].lstrip('.')

# Check if videos need to be downsampled
downsample_flag = check_for_downsample(video_path_list)

if downsample_flag:
	video_path_list = downsample_videos(video_path_list)

# Create ModelZoo project
config_path, train_config_path = deeplabcut.create_pretrained_project(
    project_name,
    your_name,
    video_path_list[0:1],
    videotype=videotype,
    model=model2use,
    analyzevideo=True,
    createlabeledvideo=True,
    copy_videos=False, # must leave copy_videos=True
)

Checking frame size...
   Pixel width x height: 320x240
No need to downsample videos
Created "/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/videos"
Created "/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/labeled-data"
Created "/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/training-datasets"
Created "/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/dlc-models"
Attempting to create a symbolic link of the video ...
Created the symlink of /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230628_Aragorn/230628_Aragorn_choice_Cam1_1.mp4 to /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/videos/230628_Aragorn_choice_Cam1_1.mp4
/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/

/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/dlc-models/iteration-0/230628_AragornJul5-trainset95shuffle1/train/pose_cfg.yaml
Analyzing video...
Using snapshot-1030000 for model /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/dlc-models/iteration-0/230628_AragornJul5-trainset95shuffle1


2023-07-05 11:52:26.727214: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Analyzing all the videos in the directory...
Starting to analyze %  /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/videos/230628_Aragorn_choice_Cam1_1.mp4
Loading  /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/videos/230628_Aragorn_choice_Cam1_1.mp4
Duration of video [s]:  5.8 , recorded with  30.0 fps!
Overall # of frames:  174  found with (before cropping) frame dimensions:  320 240
Starting to extract posture


100%|██████████| 174/174 [00:03<00:00, 49.79it/s]


Saving results in /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/videos...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
Analyzing all the videos in the directory...
Filtering with median model /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/videos/230628_Aragorn_choice_Cam1_1.mp4
Saving filtered csv poses!
Plotting results...
Analyzing all the videos in the directory...
Starting to process video: /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/videos/230628_Aragorn_choice_Cam1_1.mp4
Loading /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/videos/230628_Arag

100%|██████████| 174/174 [00:00<00:00, 286.18it/s]


Analyzing all the videos in the directory...
Loading  /Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05/videos/230628_Aragorn_choice_Cam1_1.mp4 and data.
Plots created! Please check the directory "plot-poses" within the video directory


In [4]:
# Delete the first video in the config file to rerun with new config
clean_pretrained_project(config_path)

Deleting files for rerun...
  Deleted: 230628_Aragorn_choice_Cam1_1DLC_resnet50_230628_AragornJul5shuffle1_1030000.csv
  Deleted: 230628_Aragorn_choice_Cam1_1DLC_resnet50_230628_AragornJul5shuffle1_1030000_filtered.csv
  Deleted: 230628_Aragorn_choice_Cam1_1DLC_resnet50_230628_AragornJul5shuffle1_1030000_filtered_labeled.mp4
  Deleted: 230628_Aragorn_choice_Cam1_1DLC_resnet50_230628_AragornJul5shuffle1_1030000_filtered.h5
  Deleted: 230628_Aragorn_choice_Cam1_1DLC_resnet50_230628_AragornJul5shuffle1_1030000.h5
  Deleted: 230628_Aragorn_choice_Cam1_1DLC_resnet50_230628_AragornJul5shuffle1_1030000_meta.pickle
Done deleting file.


In [5]:
# Updating the configs within the config.yaml file
edits = {
    'dotsize': 3,  # size of the dots!
    'colormap': 'rainbow',  # any matplotlib colormap
    'alphavalue': 0.2, # transparency of labels
    'pcutoff': 0.5,  # the higher the more conservative the plotting!
    'skeleton': 
         # Right Eye
        [['RightEye_Top', 'RightEye_Inner'], 
         ['RightEye_Inner', 'RightEye_Bottom'],
         ['RightEye_Outer', 'RightEye_Bottom'],
         ['RightEye_Top', 'RightEye_Outer'], 
         # Left Eye
         ['LeftEye_Top', 'LeftEye_Inner'],
         ['LeftEye_Inner', 'LeftEye_Bottom'],
         ['LeftEye_Outer', 'LeftEye_Bottom'],
         ['LeftEye_Top', 'LeftEye_Outer'],
        #  # Top of Head Counter-Clockwise to Lip
        #  ['HeadTop_Mid', 'OutlineRight_Mouth'],
        #  ['OutlineRight_Mouth', 'RightNostrils_Bottom'],
        #  ['RightNostrils_Bottom', 'UpperLip_Centre'],
        #  # Lip Counter-Clockwise to Top of Head
        #  ['UpperLip_Centre', 'OutlineLeft_Mouth'],
        #  ['OutlineLeft_Mouth', 'LeftNostrils_Bottom'],
        #  ['LeftNostrils_Bottom', 'HeadTop_Mid'],
        ],
    'skeleton_color': 'white'
}
deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)

{'Task': '230628_Aragorn', 'scorer': 'rahim', 'date': 'Jul5', 'multianimalproject': False, 'identity': None, 'project_path': '/Users/rahimhashim/Desktop/Monkey-Behavior/dlc_primate/230628_Aragorn-rahim-2023-07-05', 'video_sets': {'/Users/rahimhashim/Library/CloudStorage/GoogleDrive-rh2898@columbia.edu/.shortcut-targets-by-id/1weRx7ojG3amil91WgRMeTUVxse__Rsyt/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230628_Aragorn/230628_Aragorn_choice_Cam1_1.mp4': {'crop': '0, 320, 0, 240'}}, 'bodyparts': ['HeadTop_Mid', 'RightEarTop_Join', 'RightEarTop_High', 'RightEar_Outer', 'RightEarBottom_Low', 'RightEarBottom_Join', 'RightEar_Tragus', 'OutlineTop_Mid', 'OutlineTop_Right', 'OutlineRight_Brow', 'OutlineRight_Indent', 'OutlineRight_Cheek', 'OutlineRight_Mouth', 'OutlineChin_Mid', 'OutlineLeft_Mouth', 'OutlineLeft_Cheek', 'OutlineLeft_Indent', 'OutlineLeft_Brow', 'OutlineTop_Left', 'LeftEarTop_Join', 'LeftEarTop_High', 'LeftEar_Outer', 'LeftEarBottom_Low', 'LeftEarBottom_Join', 'LeftEa

In [ ]:
video_list_subset = video_path_list[:]
# Adding new videos to the config.yaml file
deeplabcut.add_new_videos(config_path, video_list_subset, copy_videos=False, 
            coords=None, extract_frames=False
)

# Analyze specified videos
deeplabcut.analyze_videos(config_path, video_list_subset, 
            videotype, save_as_csv=True
)

# Filter predictions
deeplabcut.filterpredictions(config_path, video_list_subset, videotype=videotype)

# Create labeled videos
deeplabcut.create_labeled_video(
   config_path, video_list_subset, 
   videotype, 
   draw_skeleton=True, 
   filtered=True,
   trailpoints=5,
)

# Plot trajectories
deeplabcut.plot_trajectories(config_path, video_list_subset, videotype, filtered=True)

In [17]:
deeplabcut.plot_trajectories(config_path, video_list_subset, videotype, filtered=True)

Loading  /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230627_Bear/230627_Bear_choice_Cam1_1.mp4 and data.
Loading  /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230627_Bear/230627_Bear_choice_Cam1_4.mp4 and data.
Loading  /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230627_Bear/230627_Bear_choice_Cam1_5.mp4 and data.
Loading  /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230627_Bear/230627_Bear_choice_Cam1_8.mp4 and data.
Loading  /Users/rahimhashim/Google Drive/My Drive/Columbia/Salzman/Monkey-Training/tasks/rhAirpuff/8. Probabilistic_Reward_Airpuff_Choice/videos/230627_Bear/230627_Bear_choice_Cam1_10.mp4 and data.
Plots created!